In [10]:
import pandas as pd
from bs4 import BeautifulSoup
from pymystem3 import Mystem
import re
import nltk
import numpy as np

In [11]:
df = pd.read_csv('train.csv', sep=';')

In [14]:
import requests

In [25]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
}

response = requests.get('https://hh.ru/vacancy/12', headers=headers)

In [26]:
response.text

'<!DOCTYPE html>\n<html class="desktop" lang="ru"><!--request: 161911226189439c9146c6c0fd153fac--><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><title data-rh="" data-reactroot="">Вакансия Экономист (бухгалтер) отдела обучения SAP в Москве, работа в компании SAP (вакансия в архиве c 14 января 2008)</title><meta name="twitter:card" content="summary_large_image"><meta name="description" content="Вакансия Экономист (бухгалтер) отдела обучения SAP в компании SAP. Зарплата: не указана. Москва. Требуемый опыт: 1–3 года. Полная занятость. Дата публикации: 13.07.2004."><meta name="keywords" content="Вакансия Экономист (бухгалтер) отдела обучения SAP в Москве, работа в компании SAP  (вакансия в архиве)"><meta property="og:image" content="https://krasnogorsk.hh.ru/employer-logo/761217.png"><meta property="og:image" content="https://i.hh.ru/logos/openGraph/hh.ru.png?v=3"><meta property="og:image:width" content="600"><

In [ ]:
response.json()

In [27]:
texts = df['text']

In [28]:
texts[0]

'<p>В молодой и дружный коллектив динамично развивающейся компании требуется Разработчик/Ведущий разработчик Oracle Siebel CRM</p> <p><strong>Требования:</strong></p> <ul> <li>Опыт работы с <strong>Oracle Siebel CRM от 2 лет</strong>;</li> <li>Обязательно знание: Siebel Tools, Runtime Events, Workflow, DVM, EAI, EIM, eScript, SQL;</li> <li>Желательные знания: Siebel Loyalty, Siebel Financial Services, CTI;</li> <li>Технический английский язык;</li> <li>Высшее техническое образование.</li> </ul> <p><strong>Обязанности:</strong></p> <ul> <li>Проектирование системы и подготовка технической документации;</li> <li>Разработка функциональности и интеграции в соответствии с требованиями;</li> <li>Оптимизация производительности системы, анализ ошибок и сбоев;</li> <li>Участие во встречах с бизнес заказчиками.</li> </ul> <p><strong>Условия:</strong></p> <ul> <li>Возможность профессионального и карьерного роста в динамично развивающейся компании;</li> <li>Заработная плата обсуждается индивидуальн

In [82]:
morph = Mystem()

In [83]:

nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

def text_to_sent(t):
    text = BeautifulSoup(t).text.lower()
    raw_sentences = tokenizer.tokenize(text.strip())
    raw_sentences = [x.split(';') for x in raw_sentences]
    raw_sentences = sum(raw_sentences, [])
    sentences = [morph.lemmatize(x) for x in raw_sentences]
    return [[y for y in x if re.match('[а-яёa-z0-9]', y)]
            for x in sentences]

[nltk_data] Downloading package punkt to
[nltk_data]     /home/petr.ermakov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [84]:
text_to_sent(texts[2])

[['федеральный',
  'сеть',
  'аптека',
  'в',
  'связь',
  'с',
  'активный',
  'расширение',
  'искать',
  'ответственный',
  'сотрудник'],
 ['работа',
  'в',
  'наш',
  'компания',
  'это',
  'возможность',
  'реализовать',
  'себя',
  'как',
  'специалист',
  'расти',
  'вместе',
  'с',
  'мы',
  'получать',
  'карьерный',
  'рост',
  'уже',
  'сейчас',
  'у',
  'мы',
  'открывать',
  'много',
  '1500',
  'аптека',
  'мы',
  'представлять',
  'в',
  '40',
  'регион',
  'у',
  'мы',
  '3',
  'формат',
  'работа',
  'чтобы',
  'соответствовать',
  'запрос',
  'разный',
  'клиент'],
 ['не',
  'упускать',
  'свой',
  'шанс',
  'начинать',
  'карьера',
  'в',
  'стабильный',
  'крупный',
  'компания'],
 ['мы',
  'предлагать',
  'стабильный',
  'работа',
  'в',
  'крупный',
  'аптечный',
  'сеть',
  'график',
  '2',
  '2',
  'или',
  '5',
  '2'],
 ['оформление', 'в', 'соответствие', 'с', 'тк', 'рф', 'с', 'первый', 'день'],
 ['социальный',
  'льгота',
  'оформление',
  'медицинский',
  'кн

In [34]:
from tqdm import tqdm
sentences = []
for text in tqdm(texts):
    sentences += text_to_sent(text)

100%|██████████| 31063/31063 [06:25<00:00, 80.50it/s] 


In [36]:
len(sentences)

343522

In [37]:
from gensim.models.word2vec import Word2Vec 

num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 24    # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

In [38]:
model.init_sims(replace=True)

In [45]:
model.most_similar(['целеустремленность']) 

/home/datagym.ta/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('инициативность', 0.9090688228607178),
 ('организованность', 0.8900696039199829),
 ('энергичность', 0.876747190952301),
 ('самостоятельность', 0.8757370710372925),
 ('трудолюбие', 0.8560793399810791),
 ('настойчивость', 0.8260329961776733),
 ('креативность', 0.8258250951766968),
 ('самоорганизованность', 0.8227561712265015),
 ('системность', 0.8213585615158081),
 ('коммуникабельность', 0.820633053779602)]

In [16]:
index2word_set = set(model.wv.index2word)

def text_to_vec(words):
    text_vec = np.zeros((300,), dtype="float32")
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model.wv[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec


texts_vecs = np.zeros((len(df['text']), 300), dtype="float32")

for i, text in enumerate(df['text']):
    s = text_to_sent(text)
    s = sum(s, [])
    texts_vecs[i] = text_to_vec(s)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    texts_vecs, df['target'],
    test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

/home/jupyterhub/jupyterhub_venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterhub/jupyterhub_venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8958150424348844

In [46]:
from sklearn.cluster import KMeans

In [48]:
word_vectors = model.wv.syn0

/home/datagym.ta/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [49]:
word_vectors.shape

(19284, 300)

In [51]:
num_clusters = 8000

In [53]:
kmeans_clustering = KMeans(n_clusters=num_clusters, n_jobs=-1, verbose=777)

In [54]:
kmeans_clustering.fit(word_vectors)

/home/datagym.ta/jupyterhub/lib/python3.6/site-packages/sklearn/cluster/_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Initialization complete
Iteration 0, inertia 3511.663330078125
Iteration 1, inertia 2485.73681640625
Iteration 2, inertia 2458.064208984375
Iteration 3, inertia 2449.107177734375
Iteration 4, inertia 2445.329833984375
Iteration 5, inertia 2443.39453125
Iteration 6, inertia 2441.866455078125
Iteration 7, inertia 2441.10546875
Iteration 8, inertia 2440.761474609375
Iteration 9, inertia 2440.38720703125
Iteration 10, inertia 2440.244384765625
Iteration 11, inertia 2440.145263671875
Iteration 12, inertia 2440.111083984375
Iteration 13, inertia 2440.090087890625
Iteration 14, inertia 2440.088134765625
Iteration 15, inertia 2440.0771484375
Iteration 16, inertia 2440.064208984375
Iteration 17, inertia 2440.027587890625
Iteration 18, inertia 2440.021728515625
Converged at iteration 18: strict convergence.
Initialization complete
Iteration 0, inertia 3521.149658203125
Iteration 1, inertia 2489.155517578125
Iteration 2, inertia 2464.325927734375
Iteration 3, inertia 2457.270263671875
Iteration 4

KMeans(n_clusters=8000, n_jobs=-1, verbose=777)

In [66]:
idx.argmax(axis=1)

array([2692, 5499, 5385, ..., 1921, 7694, 2310])

In [56]:
idx = kmeans_clustering.transform(word_vectors)

In [69]:
word_centroid_map = dict(zip( model.wv.index2word, idx.argmax(axis=1) ))

In [74]:
word_centroid_map['стрессоустойчивый']

5235

In [75]:
for cluster in range(5235, 5236):
    print("\nCluster %d" % cluster)
    
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 5235
['комфортный', 'любить', 'позитивный', 'надежный', 'стабильность', 'сплоченный', 'энергичный', 'доброжелательный', 'дружелюбный', 'внимательный', 'коммуникабельный', 'лояльный', 'трудолюбивый', 'честный', 'веселый', 'демократичный', 'разделять', 'культурный', 'дружеский', 'пунктуальный', 'добрый', 'придерживаться', 'слаженный', 'общительный', 'вокруг', 'дружелюбие', 'личность', 'комфортно', 'стрессоустойчивый', 'доверительный', 'приятно', 'уют', 'традиция', 'заряжать', 'жизнерадостный', 'раздевалка', 'кормить', 'продолжительный', 'усидчивый', 'радушный', 'радушие', 'чистоплотный', 'прочный', 'корнер']


In [96]:
def create_bag_of_centroids( wordlist, ):
    num_centroids = max( word_centroid_map.values() ) + 1
    
    bag_of_centroids = np.zeros( 8000, dtype="float32" )
    
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    return bag_of_centroids

In [ ]:
create_bag_of_centroids(sum(text_to_sent(texts[0]), [])).shape

In [ ]:
кластеризация -> фича

In [ ]:
старые фичи + новая фича -> RMSLE


In [98]:
train_centroids = np.zeros( (len(texts), num_clusters), \
    dtype="float32" )

counter = 0
for review in texts:
    sent = text_to_sent(review)
    sent = sum(sent, [])
    train_centroids[counter] = create_bag_of_centroids( sent )
    counter += 1


In [100]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_centroids, df['target'],
    test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)